## Trying out basic FIRE calculations

# Taking SIP input

In [46]:
import ipywidgets as widgets
# from IPython.display import display, VBox, HBox
from IPython.display import display
from ipywidgets import VBox, HBox 


# Create slider and manual input
sip_slider = widgets.IntSlider(
    value=20000,
    min=1000,
    max=200000,
    step=1000,
    description='Monthly SIP (₹):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='60%')
)

sip_input = widgets.BoundedIntText(
    value=20000,
    min=1000,
    max=200000,
    step=1000,
    description='Enter SIP (₹):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)

# Display output label
sip_display = widgets.Label()

# Sync slider and manual input
widgets.jslink((sip_slider, 'value'), (sip_input, 'value'))

# Update display label on change
def update_display(change):
    sip_display.value = f"👉 Your Monthly SIP is set to ₹{change['new']:,}"

sip_slider.observe(update_display, names='value')

# Display the UI
ui = VBox([
    HBox([sip_slider, sip_input]),
    sip_display
])
display(ui)

# Trigger initial display
sip_display.value = f"👉 Your Monthly SIP is set to ₹{sip_slider.value:,}"


# SIP table

In [49]:
import pandas as pd

# Get SIP value from the widget
monthly_sip = sip_slider.value
years = 30

# Prepare data for each year
data = []
for year in range(1, years + 1):
    data.append({
        "Year": year,
        "Monthly SIP (₹)": monthly_sip,
        "Yearly SIP (₹)": monthly_sip * 12
    })

# Convert to DataFrame
df_sip = pd.DataFrame(data)

# Format values
df_sip["Monthly SIP (₹)"] = df_sip["Monthly SIP (₹)"].map(lambda x: f"₹{x:,.0f}")
df_sip["Yearly SIP (₹)"] = df_sip["Yearly SIP (₹)"].map(lambda x: f"₹{x:,.0f}")

# Display
df_sip.style.set_caption("📅 Monthly & Yearly SIP Over 30 Years").set_table_styles([
    {'selector': 'caption', 'props': [('caption-side', 'top'), ('font-size', '16px'), ('font-weight', 'bold')]}
])


,Year,Monthly SIP (₹),Yearly SIP (₹)
0,1,"₹80,000","₹960,000"
1,2,"₹80,000","₹960,000"
2,3,"₹80,000","₹960,000"
3,4,"₹80,000","₹960,000"
4,5,"₹80,000","₹960,000"
5,6,"₹80,000","₹960,000"
6,7,"₹80,000","₹960,000"
7,8,"₹80,000","₹960,000"
8,9,"₹80,000","₹960,000"
9,10,"₹80,000","₹960,000"


### Display lumpsum cell


In [51]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import VBox, HBox

# Input: Lumpsum amount widget
lumpsum_input = widgets.IntText(
    value=300000,
    description='Total Lumpsum (₹):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)
display(lumpsum_input)

# ROI Input Widget
roi_input = widgets.FloatSlider(
    value=12.0,
    min=5.0,
    max=15.0,
    step=0.1,
    description='Expected ROI (%):',
    readout_format='.1f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

display(roi_input)



IntText(value=300000, description='Total Lumpsum (₹):', layout=Layout(width='50%'), style=DescriptionStyle(des…

FloatSlider(value=12.0, description='Expected ROI (%):', layout=Layout(width='70%'), max=15.0, min=5.0, readou…

In [52]:

# Get values
monthly_sip = sip_slider.value
total_lumpsum = lumpsum_input.value
print( total_lumpsum,'ROI:',roi_input.value)
years = 30
lumpsum_per_year = total_lumpsum / years
monthly_from_lumpsum = lumpsum_per_year / 12

# Prepare table
data = []
for year in range(1, years + 1):
    combined_monthly = monthly_sip + monthly_from_lumpsum
    yearly_sip = combined_monthly * 12
    data.append({
        "Year": year,
        "Monthly SIP (₹)": round(monthly_sip),
        "Lumpsum/month (₹)": round(monthly_from_lumpsum),
        "Combined Monthly Invest (₹)": round(combined_monthly),
        "Yearly Total Invest (₹)": round(yearly_sip)
    })

df_sip_lumpsum = pd.DataFrame(data)

# Format
for col in df_sip_lumpsum.columns[1:]:
    df_sip_lumpsum[col] = df_sip_lumpsum[col].map(lambda x: f"₹{x:,.0f}")

# Display
df_sip_lumpsum.style.set_caption("📅 SIP + Lumpsum Spread Investment Table").set_table_styles([
    {'selector': 'caption', 'props': [('caption-side', 'top'), ('font-size', '16px'), ('font-weight', 'bold')]}
])


300000 ROI: 12.0


,Year,Monthly SIP (₹),Lumpsum/month (₹),Combined Monthly Invest (₹),Yearly Total Invest (₹)
0,1,"₹80,000",₹833,"₹80,833","₹970,000"
1,2,"₹80,000",₹833,"₹80,833","₹970,000"
2,3,"₹80,000",₹833,"₹80,833","₹970,000"
3,4,"₹80,000",₹833,"₹80,833","₹970,000"
4,5,"₹80,000",₹833,"₹80,833","₹970,000"
5,6,"₹80,000",₹833,"₹80,833","₹970,000"
6,7,"₹80,000",₹833,"₹80,833","₹970,000"
7,8,"₹80,000",₹833,"₹80,833","₹970,000"
8,9,"₹80,000",₹833,"₹80,833","₹970,000"
9,10,"₹80,000",₹833,"₹80,833","₹970,000"


# Adding bonus column

In [58]:
# Parameters
monthly_sip = sip_slider.value
total_lumpsum = lumpsum_input.value
expected_annual_return = roi_input.value / 100
years = 30

# Precompute values
yearly_lumpsum = total_lumpsum
monthly_from_lumpsum = yearly_lumpsum / 12

# Track corpus
corpus = 0
data = []

for year in range(1, years + 1):
    combined_monthly = monthly_sip + monthly_from_lumpsum
    yearly_sip = combined_monthly * 12

    corpus = (corpus + yearly_sip) * (1 + expected_annual_return)

    data.append({
        "Year": year,
        "Monthly SIP (₹)": round(monthly_sip),
        "Lumpsum/month (₹)": round(monthly_from_lumpsum),
        "Yearly SIP + Lumpsum (₹)": round(yearly_sip),
        "Yearly Lumpsum (₹)": round(yearly_lumpsum),
        "Corpus (₹)": round(corpus)
    })

# Convert to DataFrame
df_corpus = pd.DataFrame(data)

# Format currency columns
columns_to_format = [
    "Monthly SIP (₹)", "Lumpsum/month (₹)", "Yearly SIP + Lumpsum (₹)", "Yearly Lumpsum (₹)", "Corpus (₹)"
]
for col in columns_to_format:
    df_corpus[col] = df_corpus[col].map(lambda x: f"₹{x:,.0f}")

# Add Corpus in Crores
df_corpus["Corpus (Cr ₹)"] = df_corpus["Corpus (₹)"].str.replace("₹", "").str.replace(",", "").astype(float) / 1e7
df_corpus["Corpus (Cr ₹)"] = df_corpus["Corpus (Cr ₹)"].map(lambda x: f"₹{x:.2f} Cr")

# Optional: drop Corpus (₹)
# df_corpus = df_corpus.drop(columns=["Corpus (₹)"])

# Display table
df_corpus.style.set_caption("📊 FIRE Projection with Yearly Lumpsum & Corpus in Cr").set_table_styles([
    {'selector': 'caption', 'props': [('caption-side', 'top'), ('font-size', '16px'), ('font-weight', 'bold')]}
])


,Year,Monthly SIP (₹),Lumpsum/month (₹),Yearly SIP + Lumpsum (₹),Yearly Lumpsum (₹),Corpus (₹),Corpus (Cr ₹)
0,1,"₹80,000","₹25,000","₹1,260,000","₹300,000","₹1,411,200",₹0.14 Cr
1,2,"₹80,000","₹25,000","₹1,260,000","₹300,000","₹2,991,744",₹0.30 Cr
2,3,"₹80,000","₹25,000","₹1,260,000","₹300,000","₹4,761,953",₹0.48 Cr
3,4,"₹80,000","₹25,000","₹1,260,000","₹300,000","₹6,744,588",₹0.67 Cr
4,5,"₹80,000","₹25,000","₹1,260,000","₹300,000","₹8,965,138",₹0.90 Cr
5,6,"₹80,000","₹25,000","₹1,260,000","₹300,000","₹11,452,155",₹1.15 Cr
6,7,"₹80,000","₹25,000","₹1,260,000","₹300,000","₹14,237,613",₹1.42 Cr
7,8,"₹80,000","₹25,000","₹1,260,000","₹300,000","₹17,357,327",₹1.74 Cr
8,9,"₹80,000","₹25,000","₹1,260,000","₹300,000","₹20,851,406",₹2.09 Cr
9,10,"₹80,000","₹25,000","₹1,260,000","₹300,000","₹24,764,775",₹2.48 Cr


In [60]:
# Lumpsum Increment Percentage Input
lumpsum_increment_input = widgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=20.0,
    step=0.5,
    description='Lumpsum Increase %/yr:',
    readout_format='.1f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

display(lumpsum_increment_input)


FloatSlider(value=0.0, description='Lumpsum Increase %/yr:', layout=Layout(width='70%'), max=20.0, readout_for…

In [63]:
# Inputs
monthly_sip = sip_slider.value
total_lumpsum = lumpsum_input.value

print("debug:",total_lumpsum)
expected_annual_return = roi_input.value / 100
lumpsum_inc_percent = lumpsum_increment_input.value / 100  # from slider
years = 30

# Step 1: Adjust base yearly lumpsum so sum over 30 years (with growth) = total_lumpsum
lumpsum_growth_factors = [(1 + lumpsum_inc_percent) ** i for i in range(years)]
adjusted_base_yearly_lumpsum = total_lumpsum / sum(lumpsum_growth_factors)

# Step 2: Loop through years and calculate values
corpus = 0
data = []

for year in range(1, years + 1):
    yearly_lumpsum = adjusted_base_yearly_lumpsum * ((1 + lumpsum_inc_percent) ** (year - 1))
    monthly_from_lumpsum = yearly_lumpsum / 12
    combined_monthly = monthly_sip + monthly_from_lumpsum
    yearly_sip = combined_monthly * 12

    corpus = (corpus + yearly_sip) * (1 + expected_annual_return)

    data.append({
        "Year": year,
        "Monthly SIP (₹)": round(monthly_sip),
        "Lumpsum/month (₹)": round(monthly_from_lumpsum),
        "Yearly SIP + Lumpsum (₹)": round(yearly_sip),
        "Yearly Lumpsum (₹)": round(yearly_lumpsum),
        "Corpus (₹)": round(corpus)
    })

# Step 3: Create DataFrame
df_corpus = pd.DataFrame(data)

# Step 4: Format ₹ values
columns_to_format = [
    "Monthly SIP (₹)", "Lumpsum/month (₹)", "Yearly SIP + Lumpsum (₹)", "Yearly Lumpsum (₹)", "Corpus (₹)"
]
for col in columns_to_format:
    df_corpus[col] = df_corpus[col].map(lambda x: f"₹{x:,.0f}")

# Step 5: Add Corpus in Cr ₹
df_corpus["Corpus (Cr ₹)"] = (
    df_corpus["Corpus (₹)"].str.replace("₹", "").str.replace(",", "").astype(float) / 1e7
).map(lambda x: f"₹{x:.2f} Cr")

# Step 6: Display
df_corpus.style.set_caption("📊 FIRE Projection with Incremental Lumpsum & Corpus in Cr ₹").set_table_styles([
    {'selector': 'caption', 'props': [('caption-side', 'top'), ('font-size', '16px'), ('font-weight', 'bold')]}
])


debug: 300000


,Year,Monthly SIP (₹),Lumpsum/month (₹),Yearly SIP + Lumpsum (₹),Yearly Lumpsum (₹),Corpus (₹),Corpus (Cr ₹)
0,1,"₹80,000",₹446,"₹965,349","₹5,349","₹1,081,191",₹0.11 Cr
1,2,"₹80,000",₹464,"₹965,563","₹5,563","₹2,292,364",₹0.23 Cr
2,3,"₹80,000",₹482,"₹965,786","₹5,786","₹3,649,128",₹0.36 Cr
3,4,"₹80,000",₹501,"₹966,017","₹6,017","₹5,168,962",₹0.52 Cr
4,5,"₹80,000",₹521,"₹966,258","₹6,258","₹6,871,446",₹0.69 Cr
5,6,"₹80,000",₹542,"₹966,508","₹6,508","₹8,778,509",₹0.88 Cr
6,7,"₹80,000",₹564,"₹966,768","₹6,768","₹10,914,710",₹1.09 Cr
7,8,"₹80,000",₹587,"₹967,039","₹7,039","₹13,307,559",₹1.33 Cr
8,9,"₹80,000",₹610,"₹967,321","₹7,321","₹15,987,865",₹1.60 Cr
9,10,"₹80,000",₹634,"₹967,613","₹7,613","₹18,990,136",₹1.90 Cr
